In [ ]:
!pip install transformers[torch]
!pip install SentencePiece


In [ ]:
# Importing required libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.cuda.amp import GradScaler, autocast


In [ ]:
scaler = GradScaler()


In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# # Creating the training function. This will be called in the main process. It is run depending on the epoch value.
# # The model is put into train mode and then we enumerate over the training loader and passed to the defined network

# def train(epoch, tokenizer, model, device, loader, optimizer):
#     model.train()
#     for _,data in enumerate(loader, 0):
#         y = data['target_ids'].to(device, dtype = torch.long)
#         y_ids = y[:, :-1].contiguous()
#         lm_labels = y[:, 1:].clone().detach()
#         lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
#         ids = data['source_ids'].to(device, dtype = torch.long)
#         mask = data['source_mask'].to(device, dtype = torch.long)

#         outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
#         loss = outputs[0]

#         if _%500==0:
#             print(f'Epoch: {epoch}, Loss:  {loss.item()}')

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()


In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _, data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype=torch.long)
        mask = data['source_mask'].to(device, dtype=torch.long)

        # Automatic Mixed Precision context
        with autocast():
            outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, labels=lm_labels)
            loss = outputs.loss

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        # Use scaler to scale the loss for backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
# Defining some key variables that will be used later on in the training
TRAIN_BATCH_SIZE = 4    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 4    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 10        # number of epochs to train (default: 10)
VAL_EPOCHS = 1
LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 256
SUMMARY_LEN = 256

In [ ]:
# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PLP Code/training_ds.csv')
df.head()

,ID,Item Number,Item Text,Summary
0,0,Item 1,Item 1. Business\nCompany Background\nThe Comp...,"**Summary:**\n\nApple Inc. designs, manufactur..."
1,1,Item 1A,"Item 1A. Risk Factors\nThe Company’s business,...",The text outlines various risk factors that co...
2,2,Item 1B,Item 1B. Unresolved Staff Comments\nNone.,The text indicates there are no unresolved sta...
3,3,Item 2,Item 2. Properties\nThe Company’s headquarters...,"As of September 25, 2021, the company's headqu..."
4,4,Item 3,Item 3. Legal Proceedings\nThe Company is subj...,The text describes legal proceedings involving...


In [ ]:
df = df.rename(columns={'Item Text': 'ctext', 'Summary':'text'})
df = df[['text','ctext']]
df.ctext = 'summarize: ' + df.ctext # add prefix "summarize: " to input indicating the task
print(df.head())

                                                text  \
0  **Summary:**\n\nApple Inc. designs, manufactur...   
1  The text outlines various risk factors that co...   
2  The text indicates there are no unresolved sta...   
3  As of September 25, 2021, the company's headqu...   
4  The text describes legal proceedings involving...   

                                               ctext  
0  summarize: Item 1. Business\nCompany Backgroun...  
1  summarize: Item 1A. Risk Factors\nThe Company’...  
2  summarize: Item 1B. Unresolved Staff Comments\...  
3  summarize: Item 2. Properties\nThe Company’s h...  
4  summarize: Item 3. Legal Proceedings\nThe Comp...  


In [ ]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state = SEED)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

FULL Dataset: (2248, 2)
TRAIN Dataset: (1798, 2)
TEST Dataset: (450, 2)


In [ ]:
# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

In [ ]:
# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=VALID_BATCH_SIZE, shuffle=False)

In [ ]:
# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session.
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


In [ ]:
# Training loop (taking around 22 mins)
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  6.310850143432617
Epoch: 1, Loss:  2.6222798824310303
Epoch: 2, Loss:  1.9968558549880981
Epoch: 3, Loss:  1.343300700187683
Epoch: 4, Loss:  1.701608419418335
Epoch: 5, Loss:  1.3912922143936157
Epoch: 6, Loss:  1.758395791053772
Epoch: 7, Loss:  1.8048529624938965
Epoch: 8, Loss:  0.7632544040679932
Epoch: 9, Loss:  0.7603359818458557


In [ ]:
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Completed 100


In [ ]:
print(final_df.head())

                                      Generated Text  \
0  text discusses various risk factors that could...   
1  Company is involved in various legal proceedin...   
2  information regarding Executive Compensation f...   
3  information regarding the security ownership o...   
4  text provides information about Principal Acco...   

                                         Actual Text  
0  The text outlines various risk factors that co...  
1  The text describes legal proceedings involving...  
2  The required information on executive compensa...  
3  The text states that the details regarding the...  
4  This section of the document provides informat...  


In [ ]:
#to evaluate the generated text using metrics like "bleu" and "rouge"
!pip install evaluate
import evaluate
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
metric = evaluate.load("bleu")
references = [ [a] for a in actuals ]
results = metric.compute(predictions=predictions, references=references, tokenizer=word_tokenize)
results

{'bleu': 0.2381867684489823,
 'precisions': [0.6269954164691007,
  0.37723264389931055,
  0.2644867414999187,
  0.2010236090473832],
 'brevity_penalty': 0.7112732584056507,
 'length_ratio': 0.745879801004409,
 'translation_length': 31635,
 'reference_length': 42413}

In [ ]:
!pip install rouge_score
metric = evaluate.load('rouge')
results = metric.compute(predictions=predictions, references=references, tokenizer=word_tokenize)
results

{'rouge1': 0.5925329805161652,
 'rouge2': 0.39564742601688035,
 'rougeL': 0.4888373576313437,
 'rougeLsum': 0.4897722951390241}

In [ ]:
model.save_pretrained('/content/drive/MyDrive/PLP Code/t5_model1_epochs_10')
tokenizer.save_pretrained('/content/drive/MyDrive/PLP Code/t5_tokenizer1_epochs_10')


('/content/drive/MyDrive/PLP Code/t5_tokenizer1_epochs_10/tokenizer_config.json',
 '/content/drive/MyDrive/PLP Code/t5_tokenizer1_epochs_10/special_tokens_map.json',
 '/content/drive/MyDrive/PLP Code/t5_tokenizer1_epochs_10/spiece.model',
 '/content/drive/MyDrive/PLP Code/t5_tokenizer1_epochs_10/added_tokens.json')